**IMPORT LIBRARY**

In [ ]:
# import thư viện
import requests
import time
import random
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from queue import Queue
import os
import csv
import json
from tqdm import tqdm
from IPython.display import clear_output
from tenacity import retry, stop_after_attempt, wait_fixed

In [ ]:
#import thư viện connect my sql
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy.dialects.mysql import insert
from sqlalchemy import Table, MetaData
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import text
from pandasql import sqldf
import pymysql

**CONNECT TO MYSQL**

In [ ]:
###create engine
#define
username = 'DT47_UIT'
password = '198198'
host = 'localhost'

DB_NAME = 'Tiki_Product'

# Create a connection object
databaseServerIP            = host
databaseUserName            = username
databaseUserPassword        = password
newDatabaseName             = DB_NAME
charSet                     = "utf8mb4"
cusrorType                  = pymysql.cursors.DictCursor

In [ ]:
def create_table(database: str, table: str, datatype: str):
    try:
        username = 'DT47_UIT'
        password = '198198'
        host = 'localhost'
        engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
        with engine.connect() as conn:
            conn.execute(text(f'DROP TABLE IF EXISTS {table};'))
            conn.execute(text(f'CREATE TABLE {table}({datatype})'))
            print('Table is created....')
    except SQLAlchemyError as e:
        print("Error while connecting to MySQL", e)

In [ ]:
connectionInstance = pymysql.connect(host=databaseServerIP, 
                                     user=databaseUserName, 
                                     password=databaseUserPassword, 
                                     db=newDatabaseName, 
                                     charset=charSet, 
                                     cursorclass=cusrorType)

In [ ]:
database_connection = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{DB_NAME}")

In [ ]:
#test connect
try:
    cursor = connectionInstance.cursor()  # tạo một đối tượng con trỏ
    cursor.execute("SELECT DATABASE()")  # thực hiện một truy vấn đơn giản để kiểm tra kết nối
    result = cursor.fetchone()  # lấy kết quả
    print(f"Connected to database: {result['DATABASE()']}")
except pymysql.OperationalError as e:
    print(f"Error connecting to the database: {e}")


**CREATE_TABLE**

In [ ]:
def create_table(database: str, table: str, datatype: str):
    username = 'DT47_UIT'
    password = '198198'
    host = 'localhost'
    engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
    with engine.connect() as conn:
        # xuat danh sach bang trong mysql
        result = conn.execute(text(f"SHOW TABLES LIKE '{table}'"))
        if result.fetchone():
            print(f'Table {table} already exists.')    #neu co thi bo qua
        else:
            try:
                conn.execute(text(f'CREATE TABLE {table}({datatype})'))     #khong co thi create
                print('Table is created....')
            except SQLAlchemyError as e:
                print("Error while connecting to MySQL", e)


In [ ]:
table = "product_id"
datatype = '''`index` INT AUTO_INCREMENT,
            id VARCHAR(255) UNIQUE, #tranh truong hop trung id
            category VARCHAR(255),
            sort    VARCHAR(255),
            PRIMARY KEY (`index`)'''
create_table("Tiki_Product",table=table, datatype=datatype)


In [ ]:
table = "temporary_table"  
datatype = '''
    id INT(255),
    master_id INT(255),
    sku varchar(255),
    all_time_quantity_sold FLOAT,
    breadcrumb_name varchar(255),
    breadcrumb_category_id INT(255),
    category_id INT(255),
    quantity_sold FLOAT,
    rating_average FLOAT,
    review_count INT(255),
    original_price INT(255),
    price INT(255),
    discount INT(255),
    discount_rate INT(255),
    list_price INT(255),
    price_usd varchar(255),
    order_count varchar(255),   
    inventory_status varchar(255),
    is_visible varchar(255),   
    product_name varchar(255),
    brand_id INT(255),
    brand_name varchar(255),
    categories_name varchar(255),
    description_short text,
    day_ago_created INT(255),
    fulfillment_type varchar(255),
    PRIMARY KEY (id)
'''
create_table("Tiki_Product", table=table, datatype=datatype)


In [ ]:
#Khai báo header
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/?src=header_tiki',
    'x-guest-token': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

***Define Parameter in tiki api***

In [ ]:
#lấy list danh sách dựa vào 5 lượt sort
sort_list = ['default', 'top_seller', 'newest', 'price,asc', 'price,desc']

In [ ]:
params = {
    'limit': '40',
    'include': 'sale-attrs,badges,product_links,brand,category,stock_item,advertisement',
    'aggregations': '1',
    'trackity_id': '70e316b0-96f2-dbe1-a2ed-43ff60419991',
    'category': '915',
    'page': '1',
    'sort':'default',
    'src': '',
    'urlKey':  '',
}

**CRAWL LIST ID PRODUCT**

In [ ]:
def fetch_product_ids(page, category, urlKey):
    product_ids = []

    for sort in sort_list:
        params['page'] = page
        params['category'] = category
        params['urlKey'] = urlKey
        params['sort'] = sort
        response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings?', headers=headers, params=params)
        
        data = None

        try:  # Try to decode the response content as JSON
            data = response.json().get('data')
        except ValueError:  # includes simplejson.decoder.JSONDecodeError
            print('Decoding JSON has failed')

        if response.status_code == 200 and data: 
            print(f'Request success on page {page} for {category}, {urlKey} with sort {sort}!!!')
            
            for record in data:
                product_ids.append({'id': record.get('id'),'category':urlKey,'sort':params['sort']})

        else:
            if response.status_code != 200:
                print(f'Request failed on page {page} for {category}, {urlKey} with sort {sort} and status code {response.status_code}.')
            elif not data:
                print(f'No more data on page {page} for {category}, {urlKey} with sort {sort}.')
    return product_ids, response.status_code, bool(data)

In [ ]:
def read_src_urlkey_from_txt(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    pairs = [line.strip().split(',') for line in lines]
    return pairs

In [ ]:
#read id category and urlkey from file text
pairs = read_src_urlkey_from_txt('./Data/Categories.txt')
pairs

In [ ]:
# Define the retry strategy
@retry(stop=stop_after_attempt(3), wait=wait_fixed(5))  # Stop after 3 attempts, wait for 5 seconds between each retry
def fetch_product_ids_with_retry(page, category, urlKey):
    return fetch_product_ids(page, category, urlKey)

for category, urlKey in pairs:
    product_id = []
    empty_data_count = 0  # Counter for empty data
    page = 1
    with ThreadPoolExecutor(max_workers=10) as executor:
        while empty_data_count < 10:
            pages = [page + i for i in range(10)]  # Create a list of pages
            futures = executor.map(fetch_product_ids_with_retry, pages, [category]*10, [urlKey]*10)
            
            for future in futures:
                result, status_code, data_exists = future
                product_id.extend(result)
                if status_code != 200 or not data_exists:
                    empty_data_count += 1
                else:
                    empty_data_count = 0  # Reset the counter if there's data
            page += 10  # Increase page by the number of workers

        # Save after finishing the entire data fetching for each pair
        df = pd.DataFrame(product_id)  # Add column when creating DataFrame
        sql = "INSERT IGNORE INTO tiki_product.product_id (id, category, sort) VALUES (%s, %s, %s)"
        try:
            # Open a new connection for each execute
            connectionInstance = pymysql.connect(host=databaseServerIP, 
                                 user=databaseUserName, 
                                 password=databaseUserPassword, 
                                 db=newDatabaseName, 
                                 charset=charSet, 
                                 cursorclass=cusrorType)

            with connectionInstance.cursor() as cursor:
                for _, row in df.iterrows():
                    cursor.execute(sql, (row['id'], row['category'], row['sort']))
                connectionInstance.commit()
        except Exception as e:
            print(f'Error when appending data: {e}')
        finally:
            # Ensure the connection is closed
            if connectionInstance and connectionInstance.open:
                connectionInstance.close()  # Clear the list to avoid saving data multiple times
        print(f'Data for {category}, {urlKey} has been saved.')

**CRAWL  PRODUCT DATA WITH A PRODUCT ID**

In [ ]:
def parser_product(json):
    d = dict()
    d['id'] = json.get('id')
    d['master_id'] = json.get('master_id')
    d['sku'] = json.get('sku')
    value = json.get('all_time_quantity_sold')
    if isinstance(value, str) and value.isdigit():
        value = int(value)
    elif value is None or value == 'None':
        value = 0  # or None, or -1, or whatever you want
    d['all_time_quantity_sold'] = value


    category_id = json.get('categories')
    if category_id is not None:
        d['category_id'] = category_id.get('id')
        d['categories_name']=category_id.get('name')

    breadcrumbs = json.get('breadcrumbs')
    if breadcrumbs:
        first_breadcrumb = breadcrumbs[0]  # Get the first item in the breadcrumbs list
        d['breadcrumb_name'] = first_breadcrumb.get('name')  # Get the 'name' of the first breadcrumb
        d['breadcrumb_category_id'] = first_breadcrumb.get('category_id') 

    quantity_sold = json.get('quantity_sold')
    if quantity_sold is not None:
        d['quantity_sold'] = quantity_sold.get('value')
    

        
    d['rating_average'] = json.get('rating_average')
    d['review_count'] = json.get('review_count')
    d['original_price'] = json.get('original_price')
    d['price'] = json.get('price')
    d['discount'] = json.get('discount')
    d['discount_rate'] = json.get('discount_rate')

    d['list_price'] = json.get('list_price')
    d['discount'] = json.get('discount')
    d['discount_rate'] = json.get('discount_rate')
    
    quantity_sold = json.get('quantity_sold')
    if quantity_sold is not None:
        d['quantity_sold'] = quantity_sold.get('value')
        
    d['review_count'] = json.get('review_count')
    d['inventory_status'] = json.get('inventory_status')
    d['is_visible'] = json.get('is_visible')
    d['product_name'] = json.get('name')
    configurable_products = json.get('configurable_products')
    if configurable_products:
        first_products = breadcrumbs[1]  # Get the first item in the breadcrumbs list
        d['breadcrumb_name'] = first_products.get('name')  # Get the 'name' of the first breadcrumb

    brand = json.get('brand')
    if brand is not None:
        d['brand_id'] = brand.get('id')
        d['brand_name'] = brand.get('name')
    
    # categories_name = json.get('categories')
    # if categories_name is not None:
    #     d['categories_name'] = json.get('name')
     
    d['description_short'] = json.get('short_description')
    #d['product_links'] = json.get('product_links')
    d['day_ago_created'] = json.get('day_ago_created')
        
    inventory = json.get('inventory')
    if inventory is not None:
        d['fulfillment_type'] = brand.get('fulfillment_type')
        #d['product_virtual_type'] = brand.get('product_virtual_type')

    d['inventory_status'] = json.get('inventory_status')    
    return d

In [ ]:
# <!-- def parser_product(product_json):
#     d = dict()
    
#     d['id'] = product_json.get('id')
#     d['master_id'] = product_json.get('master_id')
#     d['sku'] = product_json.get('sku')

#     value = product_json.get('all_time_quantity_sold')
#     if isinstance(value, str) and value.isdigit():
#         value = int(value)
#     elif value is None or value == 'None':
#         value = 0  # or None, or -1, or whatever you want
#     d['all_time_quantity_sold'] = value


#     breadcrumb = product_json.get('breadcrumbs')
#     if breadcrumb:
#         d['breadcrumb_name'] = breadcrumb[0].get('name')
#         d['breadcrumb_category_id'] = breadcrumb[0].get('category_id')

#     quantity_sold = product_json.get('quantity_sold')
#     d['quantity_sold'] = quantity_sold.get('value') if quantity_sold else None
    
#     d['rating_average'] = product_json.get('rating_average')
#     d['review_count'] = product_json.get('review_count')
#     d['original_price'] = product_json.get('original_price')
#     d['price'] = product_json.get('price')
#     d['discount'] = product_json.get('discount')
#     d['discount_rate'] = product_json.get('discount_rate')
#     d['list_price'] = product_json.get('list_price')
#     d['price_usd'] = product_json.get('price_usd')
#     d['order_count'] = product_json.get('order_count')
#     d['inventory_status'] = product_json.get('inventory_status')
#     d['is_visible'] = product_json.get('is_visible')

    
#     brand = product_json.get('brand')
#     if brand:
#         d['brand_id'] = brand.get('id')
#         d['brand_name'] = brand.get('name')
    
#     categories_name = product_json.get('categories')
#     d['categories_name'] = categories_name.get('name') if categories_name else None
    
#     d['description_short'] = product_json.get('short_description')
#     d['day_ago_created'] = product_json.get('day_ago_created')
    
#     inventory = product_json.get('inventory')
#     if inventory:
#         d['fulfillment_type'] = inventory.get('fulfillment_type')

#     return d -->

In [ ]:
def download_and_parse(pid):
    try:
        response = requests.get('https://tiki.vn/api/v2/products/{}'.format(pid), headers=headers, params=params)
        if response.status_code == 200 and response.text.strip():
            return parser_product(response.json())
        else:
            print(f"Received status code {response.status_code} for product ID {pid} or response body is empty")
            return None
    except (TypeError, ValueError) as e:  # Catch errors related to parsing JSON
        print(f"Error parsing JSON for product ID {pid}: {str(e)}")
        return None

In [ ]:
#test 
download_and_parse(197211969)

In [ ]:
def convert_object_to_string(df):
    object_columns = df.select_dtypes(include='object').columns.tolist()

    for col in object_columns:
        try:
            df.loc[df[col] == 'None', col] = '0'  # replace 'None' with '0'
            df.loc[:, col] = df[col].astype(str)
        except Exception as e:
            print(f"Could not to convert column {col} to string. Error: {e}")

In [ ]:
def insert_ignore(df, table_name, con):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(df.columns)
    placeholders = ', '.join(['%s'] * len(df.columns))
    query  = f"INSERT IGNORE INTO {table_name} ({cols}) VALUES ({placeholders})"
    cursor = con.cursor()
    try:
        cursor.executemany(query, tuples)
        con.commit()
    except Exception as e:
        con.rollback()
        raise e
    cursor.close()


In [ ]:
df = pd.read_sql_query('SELECT id FROM product_id', database_connection)
p_ids = df['id'].tolist()

# check id in temporary_table
connectionInstance = pymysql.connect(host=databaseServerIP, 
                                     user=databaseUserName, 
                                     password=databaseUserPassword, 
                                     db=newDatabaseName, 
                                     charset=charSet, 
                                     cursorclass=cusrorType)
try:
    df_exist = pd.read_sql_query('SELECT id FROM temporary_table', connectionInstance)
    existing_ids = df_exist['id'].tolist()

    # Convert existing_ids to set for faster lookup
    existing_ids_set = set(existing_ids)

    # Create a new list to store p_ids for which crawling is needed
    p_ids_to_crawl = []

    for pid in p_ids:
        if pid not in existing_ids_set:
            p_ids_to_crawl.append(pid)

    lines_printed = 0
    clear_after = 50  
    max_retries = 6  #so lan lap lai toi da nen loi connect
    retry_delay = 2  #delay 2s

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        # Submit only those pids to the executor which need to be crawled
        futures = {executor.submit(download_and_parse, pid): pid for pid in p_ids_to_crawl}

        # Init tqdm bar
        with tqdm(total=len(futures)) as pbar:
            for i, future in enumerate(concurrent.futures.as_completed(futures)):
                pid = futures[future]
                try:
                    retries = 0
                    while retries < max_retries:
                        try:
                            data = future.result()
                            if data is not None:
                                # Save data immediately after crawling
                                df_single = pd.DataFrame([data])
                                convert_object_to_string(df_single)
                                if pid not in existing_ids:  # Add this line to check before inserting
                                    try:
                                        insert_ignore(df_single, 'temporary_table', connectionInstance)
                                        existing_ids.append(pid)
                                        print('Crawled and saved {} items successfully'.format(i + 1))
                                        lines_printed += 1
                                    except Exception as e:
                                        print(f'Failed to save item {pid} due to error: {e}')
                                else:
                                    print(f'Skipped duplicate item {pid}. Item not saved to the database')
                            break  # If no exception is raised, break the loop
                        except (TypeError, ValueError):  # Catch errors related to parsing JSON
                            retries += 1
                            time.sleep(retry_delay)
                            continue

                    # Clear output after every 'clear_after' lines
                    if lines_printed >= clear_after:
                        clear_output(wait=True)
                        lines_printed = 0
                except Exception as exc:
                    print('%r generated an exception: %s' % (pid, exc))
                # Update tqdm bar
                pbar.update()
finally:
    # Close the connection
    connectionInstance.close()